# MSD Lung Nodule Segmentation

<a href="https://colab.research.google.com/github/felixpeters/lung-cancer-detection/blob/master/nbs/colab/01_MSD_Lung_Nodule_Segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup

In [1]:
!pip install "monai[nibabel,skimage,pillow,tqdm]" pytorch_lightning wandb

In [2]:
from typing import Optional
from pathlib import Path

import torch
from monai.config import print_config
import pytorch_lightning as pl

In [3]:
print_config()

MONAI version: 0.5.2
Numpy version: 1.19.2
Pytorch version: 1.8.0a0+52ea372
MONAI flags: HAS_EXT = False, USE_COMPILED = False
MONAI rev id: feb3a334b7bbf302b13a6da80e0b022a4cf75a4e

Optional dependencies:
Pytorch Ignite version: NOT INSTALLED or UNKNOWN VERSION.
Nibabel version: 3.2.1
scikit-image version: 0.15.0
Pillow version: 8.2.0
Tensorboard version: 2.4.1
gdown version: NOT INSTALLED or UNKNOWN VERSION.
TorchVision version: 0.9.0a0
ITK version: NOT INSTALLED or UNKNOWN VERSION.
tqdm version: 4.53.0
lmdb version: 1.1.1
psutil version: 5.8.0

For details about installing the optional dependencies, please visit:
    https://docs.monai.io/en/latest/installation.html#installing-the-recommended-dependencies



In [4]:
print(torch.cuda.is_available())
print(torch.cuda.device_count())

True
1


In [5]:
!nvidia-smi

Thu May 27 10:53:23 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 450.36.06    CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Quadro P4000        On   | 00000000:00:05.0 Off |                  N/A |
| 46%   29C    P5     6W / 105W |      4MiB /  8119MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [6]:
root_dir = Path("/storage/data/msd").absolute()
root_dir.mkdir(parents=True, exist_ok=True)
print(root_dir)

/storage/data/msd


## Define data module

In [7]:
import os
from typing import Sequence
from torch.utils.data import DataLoader
from monai.apps import DecathlonDataset
from monai.transforms import (
    Compose,
    LoadImaged,
    AddChanneld,
    Spacingd,
    ScaleIntensityd,
    CropForegroundd,
    RandCropByPosNegLabeld,
    ToTensord,
)
from monai.data.utils import list_data_collate

class MSDLungDataModule(pl.LightningDataModule):

  def __init__(self, 
               root_dir: Path, 
               batch_size: int = 1, 
               spacing: Sequence[float] = (1.5,1.5,2.0), 
               roi_size: Sequence[int] = [48, 48, 48],
               random_seed: int = 47,
               ):
    super().__init__()
    self.root_dir = root_dir
    self.spacing = spacing
    self.roi_size = roi_size
    self.batch_size = batch_size
    self.random_seed = random_seed
    self.train_transforms = Compose([
        LoadImaged(keys=["image", "label"]),
        AddChanneld(keys=["image", "label"]),
        Spacingd(keys=["image", "label"], pixdim=self.spacing, mode=("bilinear", "nearest")),
        ScaleIntensityd(keys=["image"]),
        CropForegroundd(keys=["image", "label"], source_key="image"),
        RandCropByPosNegLabeld(
            keys=["image", "label"], 
            label_key="label",
            spatial_size=self.roi_size,
            pos=1,
            neg=1,
            num_samples=2,
            image_key="image",
            image_threshold=0,
        ),
        ToTensord(keys=["image", "label"]),
    ])
    self.valid_transforms = Compose([
        LoadImaged(keys=["image", "label"]),
        AddChanneld(keys=["image", "label"]),
        Spacingd(keys=["image", "label"], pixdim=self.spacing, mode=("bilinear", "nearest")),
        ScaleIntensityd(keys=["image"]),
        CropForegroundd(keys=["image", "label"], source_key="image"),
        ToTensord(keys=["image", "label"]),
    ])

  def prepare_data(self):
    return

  def setup(self, stage: Optional[str] = None):
    if stage in (None, "fit"):
      self.train_data = DecathlonDataset(
          root_dir=self.root_dir, 
          task="Task06_Lung",
          transform=self.train_transforms,
          section="training",
          seed=self.random_seed,
          download=False,
          cache_rate=0.1,
          num_workers=os.cpu_count(),
      )
      self.valid_data = DecathlonDataset(
          root_dir=self.root_dir, 
          task="Task06_Lung",
          transform=self.valid_transforms,
          section="validation",
          seed=self.random_seed,
          download=False,
          cache_rate=0.1,
          num_workers=os.cpu_count(),
      )

  def train_dataloader(self):
    return DataLoader(
        self.train_data, 
        batch_size=self.batch_size, 
        shuffle=True,
        num_workers=os.cpu_count(),
        collate_fn=list_data_collate,
    )

  def val_dataloader(self):
    return DataLoader(
        self.valid_data, 
        batch_size=self.batch_size, 
        shuffle=False,
        num_workers=os.cpu_count(),
        collate_fn=list_data_collate,
    )

  def test_dataloader(self):
    pass

In [8]:
import matplotlib.pyplot as plt

def preview(item, z=None):
    plt.figure("Chest CTs with labels", (8, 4))
    img = item["image"].numpy()[0]
    label = item["label"].numpy()[0]
    z = z if not None else int(img.shape[2]/2)
    plt.subplot(1, 2, 1)
    plt.imshow(img[:,:,z], cmap="gray")
    plt.title("Image")
    plt.subplot(1, 2, 2)
    plt.imshow(label[:,:,z], cmap="gray")
    plt.title("Label")
    plt.show()

In [9]:
dm = MSDLungDataModule(root_dir)

## Define model module

In [10]:
from typing import Dict
import torch
import torch.nn as nn
from torch.optim import Adam
from monai.losses import DiceLoss
from monai.inferers import sliding_window_inference

from monai.networks.nets import BasicUNet

class NoduleSegmentationNet(pl.LightningModule):
  
  def __init__(self, model: nn.Module = None, lr: float = 1e-4, roi_size=(48, 48, 48)):
    super().__init__()
    if model:
      self.model = model
    else:
      self.model = BasicUNet(features=(16, 16, 32, 64, 128, 16))
    self.lr = lr
    self.roi_size = roi_size
    self.loss = DiceLoss(to_onehot_y=True, softmax=True)
    self.save_hyperparameters()

  def forward(self, x: torch.Tensor):
    return self.model(x)

  def training_step(self, batch: Dict, batch_idx: int):
    x, y = batch["image"], batch["label"]
    output = self.forward(x)
    loss = self.loss(output, y)
    self.log("train_loss", loss, logger=True)
    return {"loss": loss}

  def validation_step(self, batch: Dict, batch_idx: int):
    x, y = batch["image"], batch["label"]
    roi_size = self.roi_size
    outputs = sliding_window_inference(x, self.roi_size, 4, self.forward)
    loss = self.loss(outputs, y)
    self.log("val_loss", loss, logger=True)
    return {"val_loss": loss}

  def configure_optimizers(self):
    optimizer = Adam(self.model.parameters(), self.lr)
    return optimizer

In [11]:
model = NoduleSegmentationNet()

BasicUNet features: (16, 16, 32, 64, 128, 16).


## Configure experiment

In [12]:
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
import wandb

In [13]:
wandb.login()

wandb: Currently logged in as: felixpeters (use `wandb login --relogin` to force relogin)


True

In [14]:
logger = WandbLogger(project="lung-cancer-detection", job_type="training")

In [15]:
es = EarlyStopping(monitor="val_loss", verbose=True)
mc = ModelCheckpoint(monitor="val_loss", filename="{epoch}-{step}-{val_loss:.4f}", verbose=True, save_top_k=1)
callbacks = [es, mc]

In [16]:
torch.cuda.empty_cache()

In [17]:
from pytorch_lightning import Trainer

trainer = Trainer(
    logger=logger, 
    callbacks=callbacks,
    max_epochs=100,
    max_time={"hours": 10},
    gpus=1,
    auto_select_gpus=True,
)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores


In [18]:
trainer.fit(model, datamodule=dm)

Loading dataset: 100%|██████████| 1/1 [00:03<00:00,  3.22s/it]
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



  | Name  | Type      | Params
------------------------------------
0 | model | BasicUNet | 1.4 M 
1 | loss  | DiceLoss  | 0     
------------------------------------
1.4 M     Trainable params
0         Non-trainable params
1.4 M     Total params
5.755     Total estimated model params size (MB)


Epoch 0:  81%|████████  | 51/63 [01:34<00:22,  1.85s/it, loss=0.685, v_num=9ytj]
Validating: 0it [00:00, ?it/s]
Validating: 100%|██████████| 12/12 [00:40<00:00,  2.76s/it]

Metric val_loss improved. New best score: 0.693


Epoch 0: 100%|██████████| 63/63 [02:15<00:00,  2.15s/it, loss=0.685, v_num=9ytj]
                                                           

...

Epoch 83:  81%|████████  | 51/63 [01:19<00:18,  1.55s/it, loss=0.284, v_num=9ytj]
Validating: 0it [00:00, ?it/s]
Validating: 100%|██████████| 12/12 [00:41<00:00,  2.78s/it]

Monitored metric val_loss did not improve in the last 3 records. Best score: 0.494. Signaling Trainer to stop.


Epoch 83: 100%|██████████| 63/63 [02:00<00:00,  1.92s/it, loss=0.284, v_num=9ytj]
                                                           

Epoch 83, global step 4283: val_loss was not in top 1


Epoch 83: 100%|██████████| 63/63 [02:00<00:00,  1.92s/it, loss=0.284, v_num=9ytj]


In [19]:
wandb.finish()

train_loss,0.27743
epoch,83
trainer/global_step,4283
_runtime,10372
_timestamp,1622123197
_step,168
val_loss,0.49809


train_loss,██▇▇▇▇▇▇▇▄▆▇▆▆▆▆▆▆▃▆▆▆▆▄▆▅▃▅▃▅▄▃▅▂▁▄▄▄▆▃
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
trainer/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
_runtime,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_timestamp,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
val_loss,█▇▇▆▆▆▆▅▅▅▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁
